# Demonstrate Seg-Grad-CAM on a U-Net with a backbone from 'segmentation_models' trained on Cityscapes

## 0. Imports 

In [7]:
import seggradcam

In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os

from seggradcam.dataloaders import Cityscapes
#from metrics import iou_coef, dice_coef, dice_loss
from seggradcam.unet import csbd_unet, manual_unet, TrainUnet
from seggradcam.training_write import TrainingParameters, TrainingResults
from seggradcam.training_plots import plot_predict_and_gt, plot_loss, plot_metric
from seggradcam.seggradcam import SegGradCAM, SuperRoI, ClassRoI, PixelRoI, BiasRoI
#import matplotlib.pyplot as plt
from seggradcam.visualize_sgc import SegGradCAMplot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from seggradcam import *

## 1. Dataset set up

## 1.1 A. Prepare dataset

## 1.1 B. Load prepared dataset

## 1.2. Take a look at the images

## 2. U-Net set up

## 2.1 A. Train a U-Net yourself 

### Training plots

## 2.1 B. Load a U-Net 

## 2.2. Take a look at predictions

# 3. Seg-Grad-CAM

## 3.1 Set up layers for propagation of gradients

## 3.2 Choose an image

## 3.3 A. Seg-Grad-CAM for a pixel

## 3.3 B. Seg-Grad-CAM for the class (all pixels that were predicted as class 'cls')

## 3.3 C. Seg-Grad-CAM for a region of interest
### 3.3.C.1 for the largest set of connected pixels predicted as 'cls'

### 3.3.C.2 for the smallest set of connected pixels predicted as 'cls'

### 3.3.C.3 It is also possible to define an arbitrary RoI